In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import sounddevice as sd
import queue
import csv
import os
import json
from datetime import datetime
from collections import deque, Counter
import winsound  # Windows beep sound

print("🚀 Advanced ExamProctor v2.5 - TOTAL VIOLATION COUNTING!")

CONFIG = {
    "blink_ear_thresh": 0.25,
    "audio_thresh_db": -20.0,  # 🔊 Changed to 20dB as requested
    "mouth_mar_thresh": 0.55,
    "yaw_thresh": 15.0,
    "pitch_thresh": 12.0,
    "occlusion_frames": 15,
    "mobile_size_min": 3000,
    "mobile_size_max": 25000,
    "shoulder_move_thresh": 25,
    "total_violations_warning": 2,   # ⚠️ WARNING after 2 TOTAL violations
    "total_violations_lockout": 4    # 🔒 LOCKOUT after 4 TOTAL violations
}

class AdvancedExamProctor:
    def __init__(self, cam_idx=0, log_csv="proctor_log.csv"):
        self.log_csv_path = log_csv
        self.cam_idx = cam_idx
        self.session_start = time.time()
        self.json_alert_id = 0
        self.last_beep_time = 0
        self.exam_locked = False
        self.lockout_time = 0
        
        # 🔥 NEW: Total violation counter (ALL types combined)
        self.total_violations = 0
        
        # MediaPipe
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False, max_num_faces=4, refine_landmarks=True,
            min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.mp_draw = mp.solutions.drawing_utils
        self.drawing_spec = self.mp_draw.DrawingSpec(thickness=1, circle_radius=1)
        
        # State
        self.alerts_json = []
        self.suspicion_score = 0.0
        self.violation_history = Counter()
        self.blink_history = deque(maxlen=300)
        self.occlusion_count = 0
        self._audio_rms_db = -120.0
        self._audio_queue = queue.Queue()
        self._audio_running = False
        self.violation_cooldowns = {}
        self.fps_history = deque(maxlen=60)
        self.shoulder_positions = deque(maxlen=10)
        
        # 🔥 Warning system
        self.warning_active = False
        self.warning_message = ""
        self.warning_start_time = 0
        
        self.cap = cv2.VideoCapture(cam_idx)
        self._ensure_log_exists()
        print("✅ Proctor v2.5 READY! (Total: 2=Warning, 4=Lockout)")

    def _ensure_log_exists(self):
        if not os.path.exists(self.log_csv_path):
            with open(self.log_csv_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["timestamp", "violation_type", "value", "details"])

    def log_event(self, vtype, value="", details=""):
        now = time.time()
        if now - self.violation_cooldowns.get(vtype, 0) < 3.0: return
        self.violation_cooldowns[vtype] = now
        
        # 🔥 INCREMENT TOTAL VIOLATION COUNT
        self.total_violations += 1
        
        ts = datetime.utcnow().isoformat(timespec='seconds') + 'Z'
        with open(self.log_csv_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([ts, vtype, value, details])
        print(f"🚨 [{ts[:19]}] {vtype}: {value} (TOTAL: {self.total_violations}/4)")

    def check_total_violations(self):
        """🔥 NEW: Total violation counting (ALL types)"""
        print(f"📊 TOTAL Violations: {self.total_violations}/4")  # Debug
        
        # ⚠️ WARNING after 2 TOTAL violations
        if self.total_violations >= CONFIG["total_violations_warning"]:
            self.show_on_screen_warning(f"⚠️ WARNING: {self.total_violations} violations detected")
            self.beep_alert(1200, 500)
            
            # 🔒 LOCKOUT after 4 TOTAL violations
            if self.total_violations >= CONFIG["total_violations_lockout"]:
                self.lock_exam("Multiple violations")
                return True
                
        return False

    def beep_alert(self, frequency=800, duration=200):
        now = time.time()
        if now - self.last_beep_time > 1.0:
            try:
                winsound.Beep(frequency, duration)
                self.last_beep_time = now
            except:
                print("\a")

    def show_on_screen_warning(self, message):
        self.warning_active = True
        self.warning_message = message
        self.warning_start_time = time.time()
        print(f"🔔 {message}")

    def lock_exam(self, reason):
        if self.exam_locked: return
        self.exam_locked = True
        self.lockout_time = time.time()
        self.show_on_screen_warning(f"🔒 EXAM CANCELLED: {reason} (4+ violations)")
        self.log_event("EXAM_LOCKED", reason, f"Total {self.total_violations} violations")
        self.create_json_alert("EXAM_LOCKED", "CRITICAL", 1.0, {"reason": reason, "total_violations": self.total_violations})

    def draw_warning_overlay(self, frame, h, w):
        if self.warning_active:
            alpha = 0.3
            overlay = frame.copy()
            
            if self.exam_locked:
                cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 139), -1)
                cv2.putText(overlay, "🔒 EXAM CANCELLED", (w//2-200, h//2-50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
                cv2.putText(overlay, f"Violations: {self.total_violations}", (w//2-150, h//2+20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                cv2.putText(overlay, "Restart app to continue", (w//2-180, h//2+70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            else:
                cv2.rectangle(overlay, (0, 50), (w, 150), (0, 165, 255), -1)
                cv2.putText(overlay, self.warning_message, (20, 110), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                cv2.rectangle(overlay, (20, 120), (w-20, 140), (255, 255, 0), 3)
            
            cv2.addWeighted(overlay, alpha, frame, 1-alpha, 0, frame)
            
            if not self.exam_locked and time.time() - self.warning_start_time > 5.0:
                self.warning_active = False

    def get_current_time(self):
        now = datetime.now()
        return now.strftime("%H:%M:%S %d/%m/%Y")

    def rms_to_db(self, rms):
        return -120.0 if rms <= 1e-10 else 20.0 * math.log10(rms)

    def _audio_callback(self, indata, frames, time_info, status):
        if status: print(f"Audio: {status}")
        try:
            mono = np.mean(indata, axis=1)
            rms = np.sqrt(np.mean(mono**2))
            db = self.rms_to_db(rms)
            self._audio_queue.put_nowait(db)
        except: pass

    def start_audio(self):
        try:
            self._audio_stream = sd.InputStream(samplerate=16000, blocksize=1024, 
                                              channels=1, callback=self._audio_callback)
            self._audio_stream.start()
            self._audio_running = True
            print("🔊 Audio ON (20dB threshold)")
        except Exception as e:
            print(f"Audio optional: {e}")

    def extract_landmark_points(self, landmarks, indices, img_shape):
        h, w = img_shape[:2]
        points = []
        for idx in indices:
            lm = landmarks[idx]
            x, y = int(lm.x * w), int(lm.y * h)
            points.append(np.array([x, y], dtype=np.float32))
        return np.array(points)

    def eye_aspect_ratio(self, eye_pts):
        A = np.linalg.norm(eye_pts[1] - eye_pts[5])
        B = np.linalg.norm(eye_pts[2] - eye_pts[4])
        C = np.linalg.norm(eye_pts[0] - eye_pts[3])
        return (A + B) / (2.0 * C)

    def detect_blink(self, landmarks, img_shape):
        left_eye_indices = [36, 37, 38, 39, 40, 41]
        right_eye_indices = [42, 43, 44, 45, 46, 47]
        
        left_eye_pts = self.extract_landmark_points(landmarks, left_eye_indices, img_shape)
        right_eye_pts = self.extract_landmark_points(landmarks, right_eye_indices, img_shape)
        
        left_ear = self.eye_aspect_ratio(left_eye_pts)
        right_ear = self.eye_aspect_ratio(right_eye_pts)
        ear = (left_ear + right_ear) / 2.0
        
        self.blink_history.append(ear)
        return ear < CONFIG["blink_ear_thresh"], left_ear, right_ear

    def draw_eye_landmarks(self, frame, landmarks, h, w):
        """🔥 SMALLER RED DOTS - Only key eye points"""
        # Only draw 6 key points per eye (much smaller coverage)
        left_eye_key = [36, 39, 42]  # Top, center, bottom of left eye
        right_eye_key = [33, 45, 46]  # Top, center, bottom of right eye
        
        # Left eye - smaller dots
        for idx in left_eye_key:
            x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
            eye_color = (0, 0, 255) if landmarks[idx].y > 0.25 else (0, 255, 0)
            cv2.circle(frame, (x, y), 1, eye_color, -1)  # Radius 1 (much smaller!)
        
        # Right eye - smaller dots
        for idx in right_eye_key:
            x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
            eye_color = (0, 0, 255) if landmarks[idx].y > 0.25 else (0, 255, 0)
            cv2.circle(frame, (x, y), 1, eye_color, -1)  # Radius 1

    def detect_mobile_phone(self, frame, gray):
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if CONFIG["mobile_size_min"] < area < CONFIG["mobile_size_max"]:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                if 1.8 < aspect_ratio < 3.0 or 0.33 < aspect_ratio < 0.55:
                    peri = cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
                    
                    if len(approx) >= 4:
                        roi = frame[y:y+h, x:x+w]
                        if roi.size > 0:
                            mean_color = np.mean(roi)
                            if 50 < mean_color < 180:
                                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
                                cv2.putText(frame, "📱 MOBILE!", (x, y-10), 
                                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                                return True
        return False

    def detect_shoulder_movement(self, landmarks, img_shape):
        h, w = img_shape[:2]
        left_shoulder_idx = 10
        right_shoulder_idx = 390
        
        try:
            left_pos = np.array([landmarks[left_shoulder_idx].x * w, landmarks[left_shoulder_idx].y * h])
            right_pos = np.array([landmarks[right_shoulder_idx].x * w, landmarks[right_shoulder_idx].y * h])
            shoulder_center = (left_pos + right_pos) / 2
            
            if len(self.shoulder_positions) > 0:
                prev_center = self.shoulder_positions[-1]
                movement = np.linalg.norm(shoulder_center - prev_center)
                if movement > CONFIG["shoulder_move_thresh"]:
                    self.shoulder_positions.append(shoulder_center)
                    return True, movement
            
            self.shoulder_positions.append(shoulder_center)
        except:
            pass
        return False, 0

    def estimate_head_pose(self, img_shape, landmarks_2d):
        model_points = np.array([
            [0.0, 0.0, 0.0], [0.0, -63.6, -12.5], [-43.3, 32.7, -26.0],
            [43.3, 32.7, -26.0], [-28.9, -28.9, -24.1], [28.9, -28.9, -24.1]
        ], dtype="double")
        
        try:
            image_points = np.array(list(landmarks_2d.values()), dtype="double")
        except: return None, None, None
        
        h, w = img_shape[:2]
        focal = w
        center = (w/2, h/2)
        K = np.array([[focal,0,center[0]], [0,focal,center[1]], [0,0,1]], dtype="double")
        
        success, rvec, tvec = cv2.solvePnP(model_points, image_points, K, None)
        if not success: return None, None, None
        
        R, _ = cv2.Rodrigues(rvec)
        sy = math.sqrt(R[0,0]**2 + R[1,0]**2)
        if sy < 1e-6: return 0, 0, 0
        
        yaw = math.degrees(math.atan2(-R[2,0], sy))
        pitch = math.degrees(math.atan2(R[2,1], R[2,2]))
        roll = math.degrees(math.atan2(R[1,0], R[0,0]))
        return yaw, pitch, roll

    def create_json_alert(self, vtype, severity, conf, evidence):
        self.json_alert_id += 1
        alert = {
            "session_id": f"exam_{int(self.session_start)}",
            "alert_id": f"A{self.json_alert_id:03d}",
            "timestamp": datetime.utcnow().isoformat() + 'Z',
            "severity": severity,
            "violation_type": vtype,
            "confidence": conf,
            "suspicion_score": self.suspicion_score,
            "total_violations": self.total_violations,  # 🔥 NEW
            "evidence": evidence
        }
        self.alerts_json.append(alert)
        return alert

    def compute_mouth_aspect_ratio(self, mouth_pts):
        top, bottom, left, right = [np.array(v) for v in mouth_pts.values()]
        vert = np.linalg.norm(top - bottom)
        horiz = np.linalg.norm(left - right)
        return vert / horiz if horiz != 0 else 0.0

    def run(self):
        self.start_audio()
        print("🎥 LIVE! q=quit, s=summary")
        print("📱 Test: Any violation 2x=⚠️WARNING, 4x=🔒LOCKOUT (TOTAL COUNT)")
        
        try:
            while True:
                if self.exam_locked and time.time() - self.lockout_time > 10:
                    print("🔒 Exam locked. Restart to continue.")
                    break
                    
                frame_start = time.time()
                ret, frame = self.cap.read()
                if not ret: break
                
                frame = cv2.flip(frame, 1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = self.face_mesh.process(frame_rgb)
                h, w = frame.shape[:2]
                
                current_time = self.get_current_time()
                
                # Audio
                while not self._audio_queue.empty():
                    self._audio_rms_db = self._audio_queue.get_nowait()
                
                violations = []
                
                # 🔥 WARNING OVERLAY FIRST
                self.draw_warning_overlay(frame, h, w)
                
                if self.exam_locked:
                    cv2.imshow("🚀 Exam Proctor v2.5 - LOCKED", frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'): break
                    continue
                
                # 🔥 ALL DETECTIONS (with TOTAL violation counting)
                if self.detect_mobile_phone(frame, gray):
                    self.log_event("MobilePhone", "0.92", "Mobile detected")
                    self.create_json_alert("MobilePhone", "CRITICAL", 0.92, {"type": "phone"})
                    self.beep_alert(1000, 300)
                    self.check_total_violations()
                
                # Occlusion
                if np.mean(frame) < 20:
                    self.occlusion_count += 1
                    if self.occlusion_count > CONFIG["occlusion_frames"]:
                        self.log_event("WebcamOccluded", "1.0", "Camera blocked")
                        self.check_total_violations()
                else:
                    self.occlusion_count = 0
                
                # Face analysis
                if not results.multi_face_landmarks:
                    self.log_event("NoFace", "0", "No face")
                    self.check_total_violations()
                else:
                    faces = results.multi_face_landmarks
                    if len(faces) > 1:
                        self.log_event("MultipleFaces", str(len(faces)), f"{len(faces)} faces")
                        self.check_total_violations()
                    
                    face_lms = faces[0]
                    self.mp_draw.draw_landmarks(frame, face_lms, self.mp_face_mesh.FACEMESH_CONTOURS,
                                               connection_drawing_spec=self.drawing_spec)
                    
                    lm_list = face_lms.landmark
                    def idx(i): return lm_list[i]
                    
                    # 🔥 EYES (smaller dots)
                    self.draw_eye_landmarks(frame, lm_list, h, w)
                    
                    # 🔥 SHOULDERS
                    shoulder_violation, shoulder_move = self.detect_shoulder_movement(lm_list, frame.shape)
                    if shoulder_violation:
                        self.log_event("ShoulderMovement", f"{shoulder_move:.0f}px", "Shoulder movement")
                        self.check_total_violations()
                    
                    # Head pose
                    landmarks_2d = {
                        'nose_tip': (idx(1).x*w, idx(1).y*h),
                        'chin': (idx(152).x*w, idx(152).y*h),
                        'left_eye': (idx(33).x*w, idx(33).y*h),
                        'right_eye': (idx(263).x*w, idx(263).y*h),
                        'left_mouth': (idx(61).x*w, idx(61).y*h),
                        'right_mouth': (idx(291).x*w, idx(291).y*h)
                    }
                    yaw, pitch, roll = self.estimate_head_pose(frame.shape, landmarks_2d)
                    
                    if yaw is not None and abs(yaw) > CONFIG["yaw_thresh"]:
                        dir_str = "LookingLeft" if yaw < 0 else "LookingRight"
                        self.log_event(dir_str, f"{yaw:.1f}", f"Head turned")
                        self.check_total_violations()
                    
                    # Mouth
                    mouth_pts = {
                        'top': (idx(13).x, idx(13).y),
                        'bottom': (idx(14).x, idx(14).y),
                        'left': (idx(61).x, idx(61).y),
                        'right': (idx(291).x, idx(291).y)
                    }
                    mar = self.compute_mouth_aspect_ratio(mouth_pts)
                    if mar > CONFIG["mouth_mar_thresh"]:
                        self.log_event("MouthOpen", f"{mar:.2f}", "Talking")
                        self.check_total_violations()
                    
                    # Blink
                    if len(self.blink_history) >= 60:
                        recent_blinks = sum(1 for e in list(self.blink_history)[-60:] if e < CONFIG["blink_ear_thresh"])
                        rate = (recent_blinks / 2.0) * 60
                        if rate > 40:
                            self.log_event("ExcessiveBlinking", f"{rate:.0f}/min", "Suspicious")
                            self.check_total_violations()
                
                # 🔥 AUDIO (20dB threshold as requested)
                if self._audio_rms_db > CONFIG["audio_thresh_db"]:
                    self.log_event("AudioDetected", f"{self._audio_rms_db:.1f}dB", "Voice/Noise >-96dB")
                    self.check_total_violations()
                
                # Suspicion score
                if self.total_violations > 0:
                    self.suspicion_score = min(1.0, self.suspicion_score + 0.1)
                
                # 🔥 UI - Shows TOTAL violations
                color = (0,255,0) if self.total_violations < 2 else (0,165,255) if self.total_violations < 4 else (0,0,255)
                status = f"TOTAL: {self.total_violations}/4 | S:{self.suspicion_score:.2f}"
                
                cv2.rectangle(frame, (10,10), (w-10,80), (30,30,30), -1)
                cv2.putText(frame, status, (20,35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                cv2.putText(frame, f"Alerts: {len(self.alerts_json)}", (20,55), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
                
                cv2.putText(frame, f"A:{self._audio_rms_db:.0f}dB", (w-200,35), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0) if self._audio_rms_db < CONFIG["audio_thresh_db"] else (0,0,255), 2)
                
                cv2.rectangle(frame, (w-220, h-60), (w-10, h-10), (20,20,20), -1)
                cv2.putText(frame, current_time, (w-210, h-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
                
                cv2.imshow("🚀 Exam Proctor v2.5 - TOTAL VIOLATION COUNTING", frame)
                
                fps = 1.0 / (time.time() - frame_start)
                self.fps_history.append(fps)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q') or key == 27: break
                if key == ord('s'): self.export_summary()
                
        finally:
            self.cleanup()
            self.export_summary()

    def export_summary(self):
        duration = time.time() - self.session_start
        summary = {
            "session_id": f"exam_{int(self.session_start)}",
            "duration_min": round(duration/60, 1),
            "total_violations": self.total_violations,  # 🔥 NEW
            "total_alerts": len(self.alerts_json),
            "suspicion_score": self.suspicion_score,
            "exam_locked": self.exam_locked,
            "violations": dict(self.violation_history),
            "avg_fps": sum(self.fps_history)/len(self.fps_history) if self.fps_history else 0,
            "recommendation": "PASS" if self.total_violations < 2 and not self.exam_locked else "FAIL"
        }
        with open("session_summary.json", "w") as f:
            json.dump(summary, f, indent=2)
        print("\n📊 SUMMARY SAVED:")
        print(json.dumps(summary, indent=2))

    def cleanup(self):
        if self.cap: self.cap.release()
        cv2.destroyAllWindows()
        if self._audio_running:
            self._audio_stream.stop()
            self._audio_stream.close()

# 🔥 RUN!
if __name__ == "__main__":
    proctor = AdvancedExamProctor(cam_idx=0)
    proctor.run()


🚀 Advanced ExamProctor v2.5 - TOTAL VIOLATION COUNTING!
✅ Proctor v2.5 READY! (Total: 2=Warning, 4=Lockout)
🔊 Audio ON (20dB threshold)
🎥 LIVE! q=quit, s=summary
📱 Test: Any violation 2x=⚠️WARNING, 4x=🔒LOCKOUT (TOTAL COUNT)
🚨 [2025-12-26T01:47:55] LookingRight: 15.0 (TOTAL: 1/4)
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
📊 TOTAL Violations: 1/4
🚨 [2025-12-26T01:47:58] LookingRight: 15.3 (TOTAL: 2/4)
📊 TOTAL Violations: 2/4
🔔 ⚠️ WARNING: 2 violations detected
🚨 [2025-12-26T01:47:58] LookingLeft: -15.1 (TOTAL: 3/4)
📊 TOTAL Violations: 3/4
🔔 ⚠️ WARNING: 3 violations detected
📊 TOTAL Violations: 3/4
🔔 ⚠️ WARNING: 3 violations detected
📊 TOTAL Violations: 3/4
🔔 ⚠️ WARNING: 3 violations detected
📊 TOTAL Violations: 3/4
🔔 ⚠️ WARNING: 3 violations detected
📊 TOTAL Violations: 3/4
🔔 ⚠️ WARNING: 3 violations detected
📊 TOTAL Violations: 3/4
🔔 ⚠️ WAR